<a href="https://colab.research.google.com/github/jotramon/MillAI/blob/main/MainVideoDetectionYOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: clear results
from IPython.display import clear_output

In [ ]:
!pip install ultralytics
# clear_output()

In [ ]:
from time import time

import cv2
import torch
import logging
from tqdm import tqdm
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

logging.basicConfig(
    filename='object_detection.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import os

# Set YOLOv8 to quiet mode
os.environ['YOLO_VERBOSE'] = 'False'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Clases para la detección en videovigilancia

In [ ]:
class ObjectDetection:
    def __init__(self, capture_index=None, output_video_path='output.avi'):
        """Initializes an ObjectDetection instance with a given camera index and output video path."""
        self.capture_index = capture_index
        self.output_video_path = output_video_path
        self.email_sent = False

        # model information
        self.model = YOLO("yolo11n.pt",verbose=False)

        # visual information
        self.annotator = None
        self.start_time = 0
        self.end_time = 0

        # device information
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # VideoWriter for saving the output video
        self.output_writer = None

    def predict(self, im0):
        """Run prediction using a YOLO model for the input image `im0`."""
        results = self.model(im0,verbose=False)
        return results

    def display_fps(self, im0):
        """Displays the FPS on an image `im0`."""
        self.end_time = time()
        fps = 1 / round(self.end_time - self.start_time, 2)
        text = f"FPS: {int(fps)}"
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
        gap = 10
        cv2.rectangle(
            im0,
            (20 - gap, 70 - text_size[1] - gap),
            (20 + text_size[0] + gap, 70 + gap),
            (255, 255, 255),
            -1,
        )
        cv2.putText(im0, text, (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)

    def plot_bboxes(self, results, im0):
        """Plots bounding boxes on an image given detection results."""
        class_ids = []
        self.annotator = Annotator(im0, 3, results[0].names)
        boxes = results[0].boxes.xyxy.cpu()
        clss = results[0].boxes.cls.cpu().tolist()
        names = results[0].names
        for box, cls in zip(boxes, clss):
            class_ids.append(cls)
            self.annotator.box_label(box, label=names[int(cls)], color=colors(int(cls), True))
        return im0, class_ids

    def __call__(self, video_path=None):
        """Run object detection on video frames from a camera stream, plotting and showing the results."""
        cap = cv2.VideoCapture(video_path or self.capture_index)
        # self.output_video_path = video_path.split('.')[0] + '_output.mp4'
        assert cap.isOpened()
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Initialize VideoWriter
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
        fps = cap.get(cv2.CAP_PROP_FPS) # Set this according to your video
        self.output_writer = cv2.VideoWriter(self.output_video_path, fourcc, fps, (width, height))

        frame_count = 0
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        with tqdm(total=total_frames, desc='Processing frames') as pbar:
          while True:
              self.start_time = time()
              ret, im0 = cap.read()
              if not ret:
                  break

              results = self.predict(im0)
              im0, class_ids = self.plot_bboxes(results, im0)

              self.display_fps(im0)

              logging.info(f'Detected objects: {len(class_ids)}')
              # cv2.imshow("YOLO11 Detection", im0)

              # Write the frame to the output video
              self.output_writer.write(im0)
              pbar.update(1)
              frame_count += 1
              if cv2.waitKey(5) & 0xFF == 27:  # Press 'Esc' to exit
                  break
              # if frame_count == 5000:
              #   break

        cap.release()
        self.output_writer.release()  # Release the VideoWriter
        cv2.destroyAllWindows()


In [ ]:
detector = ObjectDetection("",output_video_path="")
detector()

## Método sencillo de inferencia para la detección de objetos

In [ ]:
# Load a model
model = YOLO("yolo11n.pt")  # pretrained YOLO11n model

# Run batched inference on a list of images
results = model(["image1.jpg", "image2.jpg"], stream=True)  # return a generator of Results objects

# Process results generator
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk